**NER**

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import date, timedelta

import xgboost as xgb
from xgboost import plot_importance, plot_tree
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

In [3]:
raot_cta_ner = pd.read_csv('cta_ner_standard_deviation_msa_up_to_date.csv')
#print(raot_cta_ner['ID'].nunique())
raot_cta_ner.head()

,WEEK_START,PROPERTY_ID,SURVEY_ID,SURVEY_DATE,STATE,STATE_ID,MSA,MSA_ID,SUBMARKET,SUBMARKET_ID,NER,NER_T4,UNITS,UNITS_T4
0,2019-06-10,5c952d95c401700015e59ab2,5d041324ca44d400152b657b,2019-06-14,Texas,16bc1b32-98de-427a-017e-8a80adaf1ca8,"Dallas, TX",62eb1840-3fe2-8049-5062-9646e18700e4,Northeast,20962d6bfb4c87e5593d2f3bff5e2f99,1357.640264,1329.476072,303.0,303.0
1,2014-09-15,5642c17655d27c0e003b5ae8,5642c1ce55d27c0e003b8820,2014-09-15,California,3e5ffbdd-4051-7165-7db7-21bdd14930db,"Sacramento, CA",a2a9af91-964e-a267-9046-91d8c6ed645a,Other,5414f19e0902eb35626ffd051c9249af,1498.000000,1534.078947,171.0,171.0
2,2015-02-02,568b4caeeb46290e00ffb7c3,568b4eeb0406580e009c4f65,2015-02-02,Texas,16bc1b32-98de-427a-017e-8a80adaf1ca8,"Houston, TX",5808a3f7-45f6-467e-864b-5e0f0e4a0cc1,Montrose/River Oaks,d8ac5f933b79217ff383d337fed58ee1,1710.047318,1710.047318,317.0,317.0
3,2019-12-02,5d85015fa8f87e001d7ceacf,5de558ef6e028b001da8ba45,2019-12-02,Colorado,d611adb9-a2fe-1a3a-63e5-1d8634942efa,"Denver, CO",f6c8f11a-8783-573a-8140-6f221f077529,Denver-South/Glendale,9898e672265aac9197cf49878fa0b453,1824.636363,1843.041817,275.0,275.0
4,2018-07-23,5642c18855d27c0e003b6fa6,5b56088c502f3c001307765a,2018-07-23,Arizona,60682b07-6de7-18cc-618b-dfd375ec2200,"Phoenix, AZ",3593fab5-48d8-741f-9eeb-16a13099553c,North Scottsdale/Fountain Hills,9fd2c103c3304388edf88dfa14f5f4db,1104.527272,1104.527272,330.0,330.0


In [4]:
raot_cta_ner['WEEK_START'] = pd.to_datetime(raot_cta_ner['SURVEY_DATE']).dt.date
raot_cta_ner = raot_cta_ner.sort_values(by='WEEK_START')

# select rows where date is between given start/end date (inclusive)
start_date = pd.to_datetime('1/1/2015')
end_date = pd.to_datetime('10/09/2022')

raot_cta_ner = raot_cta_ner.loc[
    (raot_cta_ner['WEEK_START'] >= start_date) & (raot_cta_ner['WEEK_START'] <= end_date)]

In [5]:
def start_week_date(survey_date):
    start_date = survey_date - timedelta(days=survey_date.weekday())
    
    return start_date

raot_cta_ner['WEEK_START'] = raot_cta_ner.apply(lambda x: start_week_date(x['WEEK_START']), axis=1)
raot_cta_ner.tail()

,WEEK_START,PROPERTY_ID,SURVEY_ID,SURVEY_DATE,STATE,STATE_ID,MSA,MSA_ID,SUBMARKET,SUBMARKET_ID,NER,NER_T4,UNITS,UNITS_T4
419356,2022-10-03,61bc8c253eb983002a3df742,63431774c008895a8abeed04,2022-10-09,Oklahoma,f558bb7f-6cec-5f40-73ee-4794adfd38c4,"Tulsa, OK",d0cca12e-9d37-6f6a-6d28-6b7843b4516c,South,94ea44d954d2d24ccf5fc500a96e0cee,736.783606,736.783606,305.0,305.0
10149,2022-10-03,5d69844e32fa94001dab46a8,6343035df4fbb65edac09950,2022-10-09,California,3e5ffbdd-4051-7165-7db7-21bdd14930db,"San Diego, CA",dfad4dd3-9313-6c18-50c1-0de9821b0d67,Downtown San Diego,64cd135117aa06404f9aae45d094e97d,3331.192488,3338.811619,426.0,426.0
507438,2022-10-03,5ce2d8145e5ddc00154f6aba,6342afacbe45b2160620ae6d,2022-10-09,Colorado,d611adb9-a2fe-1a3a-63e5-1d8634942efa,"Denver, CO",f6c8f11a-8783-573a-8140-6f221f077529,Aurora-South,96faed51c8be20004c7e485032facf96,1970.625000,1962.460647,216.0,216.0
672386,2022-10-03,5ec7cb3c31f7d70004c3e4b0,634304c1399b0a32501c8c7d,2022-10-09,Washington,29f94770-3b3e-9adc-491b-dd6422b92859,"Seattle, WA",673ebf82-2936-9c7e-1c0c-26b22ba92f0c,Everett/Mukilteo/Mill Creek,be03b445c4543ca90eebb728d573db82,2046.693002,2046.693002,443.0,443.0
804142,2022-10-03,577fdb996b930c0e009f0e74,634215d36516178da78f2068,2022-10-09,Nevada,ebf21ea4-8547-91a2-687b-b996dd6144ec,"Las Vegas, NV",389103f2-0f79-8743-35e7-38e657fe8f0d,Henderson/Southeast,735620776fd30a84dade8cdcd36d0fc4,1581.160000,1552.029999,600.0,600.0


In [6]:
raot_cta_ner = raot_cta_ner[['WEEK_START', 'PROPERTY_ID', 'STATE', 'MSA', 'SUBMARKET', 'NER_T4']]

## XGBOOST

In [7]:
def datetime_index(df, date_col):
    datetime_series = pd.to_datetime(df[date_col])
    datetime_index = pd.DatetimeIndex(datetime_series.values)
    df2 = df.set_index(datetime_index)
    df2.drop('date',axis=1,inplace=True)
    return df2

In [8]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    #df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth']]
    if label:
        y = df[label]
        return X, y
    return X

In [9]:
msa_data = pd.read_csv('msa_data.csv')
msa_data_lst = list(msa_data['MSA'])

In [10]:
subsets = dict((x, y) for x, y in raot_cta_ner.groupby(['MSA']))

for unique_val, subset in subsets.items():
    subsets[unique_val] = subset.groupby('WEEK_START').mean('NER')

**Extracting the data into a df**

In [11]:
for unique_val, subset in subsets.items():
    if unique_val in msa_data_lst:
        print(unique_val)
        print(subset.tail(25))

Atlanta, GA
                 NER_T4
WEEK_START             
2022-04-18  1790.830447
2022-04-25  1792.420025
2022-05-02  1799.222625
2022-05-09  1792.786981
2022-05-16  1798.340909
2022-05-23  1797.476905
2022-05-30  1795.387873
2022-06-06  1797.695587
2022-06-13  1790.822203
2022-06-20  1801.741054
2022-06-27  1802.779437
2022-07-04  1810.917568
2022-07-11  1821.147825
2022-07-18  1823.896249
2022-07-25  1832.489009
2022-08-01  1822.173624
2022-08-08  1818.919006
2022-08-15  1817.791517
2022-08-22  1821.154389
2022-08-29  1805.276109
2022-09-05  1809.485507
2022-09-12  1810.908119
2022-09-19  1808.574062
2022-09-26  1810.618248
2022-10-03  1806.185957
Austin, TX
                 NER_T4
WEEK_START             
2022-04-18  1650.012118
2022-04-25  1652.847896
2022-05-02  1647.729560
2022-05-09  1657.936431
2022-05-16  1658.813357
2022-05-23  1668.855399
2022-05-30  1674.916704
2022-06-06  1677.079371
2022-06-13  1683.115091
2022-06-20  1685.187463
2022-06-27  1696.844081
2022-07-04  1699.

In [30]:
column_names = ["Model", "Time", "MSA", "Predictions"]

df_xgboost = pd.DataFrame(columns = column_names)
df_xgboost

,Model,Time,MSA,Predictions


In [31]:
for unique_val, subset in subsets.items():
    if unique_val in msa_data_lst:
    
    
        column_names = ["Model", "Time", "MSA", "Predictions"]

        df = pd.DataFrame(columns = column_names)

                
        algorithm_name = 'XGBoost Model'
        
        subset['date'] = subset.index
        df_train = subset[:]
        df_train = datetime_index(df_train, 'date')

        
        prediction_period = pd.DataFrame(pd.date_range('2022-10-09', periods=25, freq='W'))
        prediction_period = prediction_period.rename(columns = {0:'date'})
        
        
        df_test = prediction_period
        df_test = datetime_index(df_test, 'date')
        
        
        
        X_train, y_train = create_features(df_train, label='NER_T4')
        X_test = create_features(df_test)
        
        
        reg = xgb.XGBRegressor(n_estimators=500)

        reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train)],
            early_stopping_rounds=50,
           verbose=False)     

        df_test['Prediction'] = reg.predict(X_test)
        df_all = pd.concat([df_train['NER_T4'], df_test['Prediction']])

                
        #print(df_test.reset_index()['index'])
        
        #df['Actuals'] =  df_test['NER_T4']
        df['Predictions'] =  df_test['Prediction']
        #df['MAPE score'] =  mape_score
        df['Time'] =  list(df_test.reset_index()['index'])
        df['Model'] =  algorithm_name
        df['MSA'] =  unique_val
        #df['yoy_change'] = 

        df_xgboost = df_xgboost.append(df, ignore_index=True)

In [35]:
df_xgboost

,Model,Time,MSA,Predictions,Hash Column
0,XGBoost Model,2022-10-09,"Atlanta, GA",1815.331055,"Atlanta, GA2022-10-09"
1,XGBoost Model,2022-10-16,"Atlanta, GA",1827.401855,"Atlanta, GA2022-10-16"
2,XGBoost Model,2022-10-23,"Atlanta, GA",1826.023560,"Atlanta, GA2022-10-23"
3,XGBoost Model,2022-10-30,"Atlanta, GA",1827.144653,"Atlanta, GA2022-10-30"
4,XGBoost Model,2022-11-06,"Atlanta, GA",1833.916260,"Atlanta, GA2022-11-06"
...,...,...,...,...,...
695,XGBoost Model,2023-02-26,"Washington, DC",2139.426758,"Washington, DC2023-02-26"
696,XGBoost Model,2023-03-05,"Washington, DC",2108.032471,"Washington, DC2023-03-05"
697,XGBoost Model,2023-03-12,"Washington, DC",2114.604248,"Washington, DC2023-03-12"
698,XGBoost Model,2023-03-19,"Washington, DC",2091.426514,"Washington, DC2023-03-19"


In [34]:
df_xgboost['Hash Column'] = df_xgboost['MSA'] + df_xgboost['Time'].astype(str)

In [36]:
df_xgboost.to_csv("df_xgboost_msa_level_FUTURE_FORECASTS.csv")